In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/26 01:49:21 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.27.116 instead (on interface eth0)
22/08/26 01:49:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 01:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/26 01:49:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_indexed')
yellow_sample = pd.read_parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')

In [4]:
yellow_sample

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,tolls_amount,total_amount,is_weekend,...,Date,Start(ET),Attendance,Win,margin_victory/loss,pickup_hour,dropoff_hour,is_weekend_binary,Win_binary,Start(ET)_NUMERIC
0,2018-12-25 09:08:51,2018-12-25 09:13:27,0.90,163,162,5.0,1.45,0.0,7.25,False,...,2018-12-25,12:00,19812,No,-14,9,9,0,0,1.0
1,2018-12-25 17:03:49,2018-12-25 17:07:48,0.83,237,141,5.0,2.00,0.0,7.80,False,...,2018-12-25,12:00,19812,No,-14,17,17,0,0,1.0
2,2018-12-25 00:01:04,2018-12-25 00:14:19,2.44,230,141,11.0,2.46,0.0,14.76,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
3,2018-12-25 00:48:10,2018-12-25 00:53:35,1.20,163,50,6.0,1.45,0.0,8.75,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
4,2018-12-25 00:29:09,2018-12-25 00:46:08,3.80,239,137,14.5,3.15,0.0,18.95,False,...,2018-12-25,12:00,19812,No,-14,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640635,2019-03-24 10:45:31,2019-03-24 10:50:46,0.80,68,90,5.5,0.00,0.0,6.80,True,...,2019-03-24,12:00,18263,No,-11,10,10,1,0,1.0
640636,2019-03-24 11:19:25,2019-03-24 11:31:21,1.86,48,170,9.5,2.50,0.0,15.80,True,...,2019-03-24,12:00,18263,No,-11,11,11,1,0,1.0
640637,2019-03-24 21:37:18,2019-03-24 21:41:17,1.02,162,107,5.5,2.20,0.0,11.00,True,...,2019-03-24,12:00,18263,No,-11,21,21,1,0,1.0
640638,2019-03-24 22:54:38,2019-03-24 22:59:38,0.69,236,141,5.0,0.70,0.0,9.00,True,...,2019-03-24,12:00,18263,No,-11,22,22,1,0,1.0


In [5]:
# standardized the x columns code from tutorial 2


from scipy.stats import zscore

x_cols = ['fare_amount','tolls_amount', 'total_amount', 'is_weekend_binary', 'trip_length', 
          'Attendance', 'Win_binary', 'margin_victory/loss','pickup_hour', 'dropoff_hour', 
          'PULocationID', 'DOLocationID', 'Start(ET)_NUMERIC']

y_cols = ['tip_amount']

df_standard = yellow_sample[x_cols].astype(float).apply(zscore)

# format output to 4 decimal places
pd.options.display.float_format = '{:,.4f}'.format
df_standard.describe().loc[['mean','std']]



,fare_amount,tolls_amount,total_amount,is_weekend_binary,trip_length,Attendance,Win_binary,margin_victory/loss,pickup_hour,dropoff_hour,PULocationID,DOLocationID,Start(ET)_NUMERIC
mean,-0.0000,-0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [6]:
# fit the lasso model code from tutorial 2

import numpy as np
from glmnet import ElasticNet

elastic_net_model_lasso = ElasticNet(alpha=1) 
elastic_net_model_lasso.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    yellow_sample[y_cols].values.flatten()
)

ElasticNet()

In [7]:
# fit the ridge model code from tutorial 2

elastic_net_model_ridge = ElasticNet(alpha=0) 
elastic_net_model_ridge.fit(
    df_standard.values, 
    # flatten the array (from 2d matrix to 1d vector) to remove the warning message:
    # A column-vector y was passed when a 1d array was expected.
    yellow_sample[y_cols].values.flatten()
)

ElasticNet(alpha=0)

In [8]:
# gets the best lambda value for both ridge and LASSO

best_lambda_lasso = elastic_net_model_lasso.lambda_best_[0]
best_lambda_ridge = elastic_net_model_ridge.lambda_best_[0]

print(f'Best lambda value for LASSO:', best_lambda_lasso)
print(f'Best lambda value for Ridge:', best_lambda_ridge)


Best lambda value for LASSO: 0.012131562052867662
Best lambda value for Ridge: 0.15308295078753603


In [9]:
# drop the tpep pickup and dropoff columns, and categorical columns which have not been indexed

yellow = yellow.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Start(ET)',
                     'Win', 'is_weekend')

In [10]:
# use all 2018 data as a train set

# then use the first game of 2019 as the test set

yellow_train = yellow.where((F.col('Date') < '2019-01-11'))

yellow_test = yellow.where((F.col('Date') == '2019-01-11'))


In [11]:
# drop Date columns for both train and test

yellow_train = yellow_train.drop('Date')

yellow_test = yellow_test.drop('Date')

In [12]:
# create a vector for train data with full columns

from pyspark.ml.feature import VectorAssembler

features = 'features'

# use all featuresn
input_cols_full = ['fare_amount','tolls_amount', 'trip_distance', 'total_amount', 'is_weekend_binary', 'trip_length', 
                   'Win_binary','PULocationID', 'DOLocationID', 'pickup_hour', 'dropoff_hour','Attendance', 
                    'margin_victory/loss', 'Start(ET)_NUMERIC']

vectorAssembler_full = VectorAssembler(
    inputCols = input_cols_full, 
    outputCol = features)


v_yellow_train_full = vectorAssembler_full.transform(yellow_train.dropna('any'))

v_yellow_train_full = v_yellow_train_full.select(['features', 'tip_amount'])

v_yellow_train_full.show(3)

22/08/26 01:49:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,0.0,2.56,22...|      4.58|
|[26.0,0.0,7.85,33...|      5.56|
|[5.0,0.0,0.9,7.25...|      1.45|
+--------------------+----------+
only showing top 3 rows



In [13]:
# creation of test vector for full column

v_yellow_test_full = vectorAssembler_full.transform(yellow_test.dropna('any'))

v_yellow_test_full = v_yellow_test_full.select(['features', 'tip_amount'])

v_yellow_test_full.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,0.0,2.88,18...|       3.7|
|(14,[0,2,3,5,7,8,...|      2.26|
|(14,[0,2,3,5,7,8,...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [14]:
# LassoM stands for LASSO model

from pyspark.ml.regression import LinearRegression

LassoM = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_lasso, elasticNetParam = 1).fit(v_yellow_train_full)

22/08/26 01:50:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/26 01:50:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [15]:
#coefficients for LASSO regression

pd.DataFrame(
    data=[LassoM.intercept] + list(LassoM.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

# based on LASSO, Attendance, margin_victory/loss, Win_binary, trip_length, PULocationID , DOLocationID and 
# Start(ET) are all irrelevant

,coefficient
intercept,-0.6962
fare_amount,-0.7411
tolls_amount,-0.7130
trip_distance,-0.0299
total_amount,0.7861
is_weekend_binary,0.0937
trip_length,0.0000
Win_binary,0.0000
PULocationID,0.0000
DOLocationID,0.0000


In [16]:
features = 'features'

# features selected after selection through LASSO
input_cols_reduced = ['fare_amount', 'trip_distance', 'total_amount', 'tolls_amount', 'is_weekend_binary',
                     'PULocationID', 'DOLocationID', 'pickup_hour','dropoff_hour', 'Start(ET)_NUMERIC']


vectorAssembler_reduced = VectorAssembler(
    inputCols = input_cols_reduced, 
    outputCol = features)


v_yellow_train_reduced = vectorAssembler_reduced.transform(yellow_train.dropna('any'))

v_yellow_train_reduced = v_yellow_train_reduced.select(['features', 'tip_amount'])

v_yellow_train_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[26.0,7.85,33.36,...|      5.56|
|[5.0,0.9,7.25,0.0...|      1.45|
+--------------------+----------+
only showing top 3 rows



In [17]:
# creating vector for the reduced columns

v_yellow_test_reduced = vectorAssembler_reduced.transform(yellow_test.dropna('any'))

v_yellow_test_reduced = v_yellow_test_reduced.select(['features', 'tip_amount'])

v_yellow_test_reduced.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|(10,[0,1,2,5,6],[...|      2.26|
|(10,[0,1,2,5,6],[...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [18]:
# lm stands for linear model

lm = LinearRegression(featuresCol='features', labelCol = 'tip_amount').fit(v_yellow_train_reduced)

22/08/26 01:50:10 WARN Instrumentation: [24cd6a33] regParam is zero, which might cause numerical instability and overfitting.


22/08/26 01:50:13 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [19]:
# coefficients of linear model

pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols_reduced,
    columns=['coefficient']
)

,coefficient
intercept,-0.8573
fare_amount,-0.8752
trip_distance,-0.0380
total_amount,0.9013
tolls_amount,-0.8673
is_weekend_binary,0.1418
PULocationID,0.0001
DOLocationID,0.0002
pickup_hour,-0.0238
dropoff_hour,0.0064


In [20]:
# training summary of linear model

trainingSummary_lm = lm.summary

print("RMSE: %f" % trainingSummary_lm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_lm.r2)
print("MSE: %f" % trainingSummary_lm.meanSquaredError)
print("MAE: %f" % trainingSummary_lm.meanAbsoluteError)


RMSE: 0.334387
r2: 0.947511
MSE: 0.111815
MAE: 0.270444


In [21]:
# prediction of the linear model

from pyspark.ml.evaluation import RegressionEvaluator


lm_predictions = lm.transform(v_yellow_test_reduced)
lm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|   4.2598461766094|       3.7|[13.0,2.88,18.5,0...|
|2.5660873917487854|      2.26|(10,[0,1,2,5,6],[...|
| 2.187651174567468|      1.86|(10,[0,1,2,5,6],[...|
|2.7277228029519716|      2.49|(10,[0,1,2,5,6],[...|
| 4.319614696434301|      4.06|(10,[0,1,2,5,6],[...|
+------------------+----------+--------------------+
only showing top 5 rows



In [22]:
# test evaluation of the linear model

test_result_lm = lm.evaluate(v_yellow_test_reduced)

print("R Squared (R2) on test data = %g" % test_result_lm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_lm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_lm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_lm.meanAbsoluteError)

R Squared (R2) on test data = 0.942329
Root Mean Squared Error (RMSE) on test data = 0.354405
Mean Squared Error (MSE) on test data = 0.125603
Mean Absolute Error (MAE) on test data = 0.290916


In [23]:
# rm stands for ridge model

rm = LinearRegression(featuresCol = 'features', labelCol = 'tip_amount', 
                      regParam = best_lambda_ridge, elasticNetParam = 0).fit(v_yellow_train_full)

In [24]:
# coefficients of the ridge model

pd.DataFrame(
    data=[rm.intercept] + list(rm.coefficients),
    index=['intercept'] + input_cols_full,
    columns=['coefficient']
)

,coefficient
intercept,0.0565
fare_amount,-0.0428
tolls_amount,-0.0145
trip_distance,-0.0382
total_amount,0.1968
is_weekend_binary,-0.0095
trip_length,-0.0041
Win_binary,-0.0003
PULocationID,0.0001
DOLocationID,0.0001


In [25]:
# training summary of ridge model

trainingSummary_rm = rm.summary

print("RMSE: %f" % trainingSummary_rm.rootMeanSquaredError)
print("r2: %f" % trainingSummary_rm.r2)
print("MSE: %f" % trainingSummary_rm.meanSquaredError)
print("MAE: %f" % trainingSummary_rm.meanAbsoluteError)


RMSE: 0.909676
r2: 0.611544
MSE: 0.827510
MAE: 0.459999


In [26]:
# predictions of ridge model

rm_predictions = rm.transform(v_yellow_test_full)
rm_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
| 2.907149076523077|       3.7|[13.0,0.0,2.88,18...|
|2.1000315567582977|      2.26|(14,[0,2,3,5,7,8,...|
|1.7522810709891417|      1.86|(14,[0,2,3,5,7,8,...|
|1.7235504015063388|      2.49|(14,[0,2,3,5,7,8,...|
| 3.709396680426896|      4.06|(14,[0,2,3,5,7,8,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [27]:
# test evaluation for ridge model

test_result_rm = rm.evaluate(v_yellow_test_full)

print("R Squared (R2) on test data = %g" % test_result_rm.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_rm.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result_rm.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_rm.meanAbsoluteError)

R Squared (R2) on test data = 0.587413
Root Mean Squared Error (RMSE) on test data = 0.947936
Mean Squared Error (MSE) on test data = 0.898583
Mean Absolute Error (MAE) on test data = 0.42234
